Menginstall kaggle untuk api dataset

In [ ]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!chmod 600 /content/kaggle.json

Mendownload dataset menggunakan api dataset kaggle

In [ ]:
!KAGGLE_CONFIG_DIR=/content kaggle datasets download -d keshavramaiah/hotel-recommendation

 57% 9.00M/15.7M [00:00<00:00, 30.3MB/s]
100% 15.7M/15.7M [00:00<00:00, 51.1MB/s]


Mengextrak dataset

In [ ]:
!unzip /content/hotel-recommendation.zip

Archive:  /content/hotel-recommendation.zip
  inflating: Hotel_Room_attributes.csv  
  inflating: Hotel_details.csv       
  inflating: hotel_price_min_max - Formula.csv  
  inflating: hotels_RoomPrice.csv    


Mengimpor library yang di butuhkan

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers.core import Dense,Activation,Dropout
from keras.layers import LSTM
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from keras.callbacks import EarlyStopping
import math
import os
import nltk

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

Membuat variabel yang di dalamnya dataset

In [65]:
hotel_details = pd.read_csv("/content/Hotel_details.csv")
hotel_rooms = pd.read_csv("/content/Hotel_Room_attributes.csv")
hotel_cost = pd.read_csv("/content/hotels_RoomPrice.csv")

melihat besaran data pada dataset dan isi dari dataset

In [66]:
hotel_details

,id,hotelid,hotelname,address,city,country,zipcode,propertytype,starrating,latitude,longitude,Source,url,curr
0,46406,1771651,Mediteran Bungalow Galeb,Vukovarska 7,Omis,Croatia,21310.0,Holiday parks,4,43.440124,16.682505,2,https://www.booking.com/hotel/hr/bungalow-luxu...,EUR
1,46407,177167,Hotel Polonia,Plac Teatralny 5,Torun,Poland,NaN,Hotels,3,53.012329,18.603800,5,https://www.agoda.com/en-gb/hotel-polonia/hote...,EUR
2,46408,1771675,Rifugio Sass Bece,"Belvedere del Pordoi,1",Canazei,Italy,38032.0,Hotels,3,46.477920,11.813350,2,http://www.booking.com/hotel/it/rifugio-sass-b...,EUR
3,46409,177168,Madalena Hotel,Mykonos,Mykonos,Greece,84600.0,Hotels,3,37.452316,25.329849,5,https://www.agoda.com/en-gb/madalena-hotel/hot...,EUR
4,46410,1771718,Pension Morenfeld,Mair im Korn Strasse 2,Lagundo,Italy,39022.0,Hotels,3,46.682780,11.131736,2,http://www.booking.com/hotel/it/pension-morenf...,EUR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108043,154449,1771620,Hotel Stadt Tuttlingen,Donaustrasse 30,Tuttlingen,Germany,78532.0,Hotels,3,47.985364,8.817228,2,http://www.booking.com/hotel/de/stadt-tuttling...,EUR
108044,154450,1771626,Hotel Beroe,15 Yasen Str.,Sveti Vlas,Bulgaria,8256.0,Hotels,3,42.710919,27.763325,2,http://www.booking.com/hotel/bg/beroe.en-gb.html,BGN
108045,154451,177163,Hotel Rothmans,14 Ohrid Str.,Bansko,Bulgaria,2770.0,Hotels,3,41.837379,23.492391,2,http://www.booking.com/hotel/bg/rothmans-bansk...,BGN
108046,154452,177164,Boutique Suites Rainbow 4,Cacao Beach,Sunny Beach,Bulgaria,8240.0,Hotels,3,42.674317,27.704325,2,https://www.booking.com/hotel/bg/rainbow-4-bou...,BGN


In [67]:
hotel_rooms

,id,hotelcode,roomamenities,roomtype,ratedescription
0,50677497,634876,Air conditioning: ;Alarm clock: ;Carpeting: ;C...,Double Room,"Room size: 15 m²/161 ft², Shower, 1 king bed"
1,50672149,8328096,Air conditioning: ;Closet: ;Fireplace: ;Free W...,Vacation Home,"Shower, Kitchenette, 2 bedrooms, 1 double bed ..."
2,50643430,8323442,Air conditioning: ;Closet: ;Dishwasher: ;Firep...,Vacation Home,"Shower, Kitchenette, 2 bedrooms, 1 double bed ..."
3,50650317,7975,Air conditioning: ;Clothes rack: ;Coffee/tea m...,Standard Triple Room,"Room size: 20 m²/215 ft², Shower, 3 single beds"
4,50650318,7975,Air conditioning: ;Clothes rack: ;Coffee/tea m...,Standard Triple Room,"Room size: 20 m²/215 ft², Shower, 3 single beds"
...,...,...,...,...,...
165868,88527729,268416,Air conditioning: ;Alarm clock: ;Carpeting: ;C...,Twin Room,"Room size: 17 m²/183 ft², Non-smoking, Shower,..."
165869,88527730,268416,Air conditioning: ;Alarm clock: ;Carpeting: ;C...,Twin Room,"Room size: 17 m²/183 ft², Non-smoking, Shower,..."
165870,88527731,268416,Air conditioning: ;Alarm clock: ;Carpeting: ;C...,Twin Room,"Room size: 17 m²/183 ft², Non-smoking, Shower,..."
165871,88527215,268416,Air conditioning: ;Free Wi-Fi in all rooms!: ;...,Twin Standard,"Pay nothing until November 21, 2019, Free Wi-F..."


In [68]:
hotel_cost

,id,refid,hotelcode,websitecode,dtcollected,ratedate,los,guests,roomtype,onsiterate,...,promoname,status_code,taxstatus,taxtype,taxamount,proxyused,israteperstay,mealinclusiontype,hotelblock,input_dtcollected
0,50677497,8646773,634876,5,2019-10-12 15:46:54,2019-11-02,1,1,Double Room,82.36,...,NaN,200,2,NaN,0.0,media:M3diAproxy@173.44.165.126:80,Y,Free Breakfast,NaN,2019-10-13
1,50672149,6234691,8328096,5,2019-10-12 15:47:45,2019-11-30,1,1,Vacation Home,636.09,...,NaN,200,2,NaN,0.0,media:M3diAproxy@173.44.164.126:80,Y,NaN,NaN,2019-10-13
2,50643430,7015677,8323442,5,2019-10-12 15:47:00,2019-12-20,1,1,Vacation Home,591.74,...,NaN,200,2,NaN,0.0,media:M3diAproxy@173.44.166.122:80,Y,NaN,NaN,2019-10-13
3,50650317,7327094,7975,5,2019-10-12 15:47:02,2019-12-28,1,1,Standard Triple Room,881.48,...,NaN,200,2,Tax 20%,0.0,media:M3diAproxy@209.242.219.141:80,Y,NaN,NaN,2019-10-13
4,50650318,7327094,7975,5,2019-10-12 15:47:02,2019-12-28,1,1,Standard Triple Room,897.53,...,NaN,200,2,VAT $ 179.51,0.0,media:M3diAproxy@209.242.219.141:80,Y,Free breakfast for {3},NaN,2019-10-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165868,88527729,9572875,268416,5,2019-10-13 01:03:10,2019-11-26,1,1,Twin Room,81.83,...,NaN,200,2,VAT $ 16.37,0.0,media:M3diAproxy@173.44.166.122:80,Y,NaN,NaN,2019-10-13
165869,88527730,9572875,268416,5,2019-10-13 01:03:10,2019-11-26,1,1,Twin Room,89.80,...,NaN,200,2,VAT $ 17.97,0.0,media:M3diAproxy@173.44.166.122:80,Y,Free breakfast for {2},NaN,2019-10-13
165870,88527731,9572875,268416,5,2019-10-13 01:03:10,2019-11-26,1,1,Twin Room,99.78,...,NaN,200,2,VAT $ 19.96,0.0,media:M3diAproxy@173.44.166.122:80,Y,Free breakfast for {2},NaN,2019-10-13
165871,88527215,9572875,268416,5,2019-10-13 01:03:11,2019-11-26,1,1,Twin Standard,70.13,...,NaN,200,2,Tax 20%,0.0,media:M3diAproxy@173.44.166.122:80,Y,NaN,NaN,2019-10-13


# **Data Preprocessing**

mendrop kolom pada dataset yang dianngap tidak relevan

In [69]:
hotel_details.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108048 entries, 0 to 108047
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            108048 non-null  int64  
 1   hotelid       108048 non-null  int64  
 2   hotelname     108048 non-null  object 
 3   address       102955 non-null  object 
 4   city          108048 non-null  object 
 5   country       108048 non-null  object 
 6   zipcode       83486 non-null   float64
 7   propertytype  108048 non-null  object 
 8   starrating    108048 non-null  int64  
 9   latitude      108048 non-null  float64
 10  longitude     108048 non-null  float64
 11  Source        108048 non-null  int64  
 12  url           107937 non-null  object 
 13  curr          108048 non-null  object 
dtypes: float64(3), int64(4), object(7)
memory usage: 11.5+ MB


In [70]:
hotel_details = hotel_details.drop(["id", "zipcode"], axis=1)
hotel_rooms = hotel_rooms.drop(["id"], axis=1)

mencari missing value pada dataset

In [71]:
hotel_details.isnull().sum()

hotelid            0
hotelname          0
address         5093
city               0
country            0
propertytype       0
starrating         0
latitude           0
longitude          0
Source             0
url              111
curr               0
dtype: int64

In [72]:
hotel_rooms.isnull().sum()

hotelcode             0
roomamenities      4819
roomtype              0
ratedescription    4819
dtype: int64

In [73]:
hotel_cost.isnull().sum()

id                        0
refid                     0
hotelcode                 0
websitecode               0
dtcollected               0
ratedate                  0
los                       0
guests                    0
roomtype                  0
onsiterate                0
netrate                   0
currency                  0
ratedescription        4819
ratetype               4819
sourceurl                 0
roomamenities          4819
maxoccupancy              0
ispromo                   0
closed                    0
discount                  0
promoname            162696
status_code               0
taxstatus                 0
taxtype                8370
taxamount                 0
proxyused                 0
israteperstay             0
mealinclusiontype     68486
hotelblock           161054
input_dtcollected         0
dtype: int64

Melakukan drop pada missing value pada dataset

In [74]:
hotel_details = hotel_details.dropna()
hotel_rooms = hotel_rooms.dropna()

Mengecek kembali missing value

In [75]:
hotel_details.isnull().sum()

hotelid         0
hotelname       0
address         0
city            0
country         0
propertytype    0
starrating      0
latitude        0
longitude       0
Source          0
url             0
curr            0
dtype: int64

In [76]:
hotel_rooms.isnull().sum()

hotelcode          0
roomamenities      0
roomtype           0
ratedescription    0
dtype: int64

mendrop nilai duplicate pada kolom hotelid

In [77]:
hotel_details.drop_duplicates(subset='hotelid',keep=False,inplace=True)

Menggabungkan dataset hotel_room dan hotel_details berdasarkan hotelcode dan hotelid

In [78]:
hotel=pd.merge(hotel_rooms,hotel_details,left_on='hotelcode',right_on='hotelid',how='inner')

mengecek kolom pada dataset yang sudah di gabung

In [79]:
hotel.columns

Index(['hotelcode', 'roomamenities', 'roomtype', 'ratedescription', 'hotelid',
       'hotelname', 'address', 'city', 'country', 'propertytype', 'starrating',
       'latitude', 'longitude', 'Source', 'url', 'curr'],
      dtype='object')

mendrop dataset yang tidak relevan

In [80]:
hotel = hotel.drop(["hotelid", "url", "curr", "Source"], axis=1)

Mengecek kembali kolom dataset

In [81]:
hotel.columns

Index(['hotelcode', 'roomamenities', 'roomtype', 'ratedescription',
       'hotelname', 'address', 'city', 'country', 'propertytype', 'starrating',
       'latitude', 'longitude'],
      dtype='object')

# **Recommender system based only on City and ratings about the hotel**

Membuat system rekomendasi berdasarkan kota dan ratomh pada hotel

In [95]:
def city(city):
    hotel['city']=hotel['city'].str.lower()
    citybase=hotel[hotel['city']==city.lower()]
    citybase=citybase.sort_values(by='starrating',ascending=False)
    citybase.drop_duplicates(subset='hotelcode',keep='first',inplace=True)
    if(citybase.empty==0):
        hname=citybase[['hotelname','starrating','address','roomamenities','ratedescription']]
        return hname.head()
    else:
        print('No Hotels Available')

Menampilkan top 5 pada kota london

In [96]:
print('Top 5')
city('London')

Top 5


,hotelname,starrating,address,roomamenities,ratedescription
48985,The Colonnade London Hotel,4,2 Warrington Crescent,"additional toilet,air conditioning,alarm clock...","Room size: 28 m²/301 ft², Balcony/terrace, Non..."
58663,South Point Suites - London Bridge,4,Bermondsey Street,"air conditioning,alarm clock,blackout curtains...","Room size: 24 m²/258 ft², Street view, Non-smo..."
106943,Doubletree by Hilton London Chelsea,4,Imperial Road Imperial Wharf,"air conditioning,alarm clock,blackout curtains...","Room size: 20 m²/215 ft², Street view, Non-smo..."
57452,Sir Christopher Wren Hotel & Spa,4,Unknown,"air conditioning,alarm clock,bathrobes,blackou...","Room size: 25 m²/269 ft², Non-smoking, Shower ..."
92479,The Manor Hotel Heathrow,4,"Village Green, Datchet,","additional bathroom,additional toilet,air cond...","Shower and bathtub, 1 double bed"


Memberikan besaran angka 1-4 pada room,semakin kecil angka nya maka semakin buruk layanan yang diberikan oleh botel

In [97]:
room_no=[('king',2),
   ('queen',2), 
    ('triple',3),
    ('master',3),
   ('family',4),
   ('murphy',2),
   ('quad',4),
   ('double-double',4),
   ('mini',2),
   ('studio',1),
    ('junior',2),
   ('apartment',4),
    ('double',2),
   ('twin',2),
   ('double-twin',4),
   ('single',1),
     ('diabled',1),
   ('accessible',1),
    ('suite',2),
    ('one',2)
   ]

Mengakumulasikan besaran angka berdasarkan roomtype 

In [98]:
def calc():
    guests_no=[]
    for i in range(hotel.shape[0]):
        temp=hotel['roomtype'][i].lower().split()
        flag=0
        for j in range(len(temp)):
            for k in range(len(room_no)):
                if temp[j]==room_no[k][0]:
                    guests_no.append(room_no[k][1])
                    flag=1
                    break
            if flag==1:
                break
        if flag==0:
            guests_no.append(2)
    hotel['guests_no']=guests_no
calc()

Membuat system rekomendasi berdasarkan roomtype dan rating

In [99]:
def citybased(city,number):
    hotel['city']=hotel['city'].str.lower()
    popbased=hotel[hotel['city']==city.lower()]
    popbased=popbased[popbased['guests_no']==number].sort_values(by='starrating',ascending=False)
    popbased.drop_duplicates(subset='hotelcode',keep='first',inplace=True)
    if popbased.empty==True:
        print('Sorry No Hotels Available\n tune your constraints')
    else:
        return popbased[['hotelname','roomtype','guests_no','starrating','address','roomamenities','ratedescription']].head(10)


Menampilkan top hotel dengan kota berdasarkan rating dan roomtype tertinggi

In [100]:
citybased("London", 3)

,hotelname,roomtype,guests_no,starrating,address,roomamenities,ratedescription
41643,J Hotel London,Executive Triple,3,4,60-62 Queensborough Terrace,"air conditioning,blackout curtains,coffee/tea ...","Shower, 1 double bed or 1 single bed"
60558,Copthorne London Gatwick,Superior Triple Room,3,4,Copthorne Way,"air conditioning,alarm clock,carpeting,closet,...","Room size: 23 m²/248 ft², Shower and bathtub, ..."
126241,Hotel Henry VIII,Triple,3,4,23 Leinster Gardens,"air conditioning,blackout curtains,closet,coff...","Room size: 14 m²/151 ft², Street view, Non-smo..."
117263,Barrington Lodge,Deluxe Triple Room,3,4,Hunter Road,"air conditioning,alarm clock,carpeting,closet,...","Room size: 30 m²/323 ft², Shower, 3 single beds"
116538,Lucky 8 Hotel,Triple Room,3,4,York Road,"air conditioning,free wi-fi in all rooms!,in-r...","Free Breakfast, Free Wi-Fi, Extra low price! (..."
66457,The Park City Grand Plaza Kensington Hotel,Executive Triple Room,3,4,18-30 Lexham Gardens,"air conditioning,alarm clock,carpeting,closet,...","Room size: 26 m²/280 ft², City view, Non-smoki..."
27377,Grange Langham Court Hotel,Triple Room,3,4,31-35 Langham Street,"air conditioning,closet,clothes rack,coffee/te...","Room size: 16 m²/172 ft², Non-smoking, Separat..."
65809,Atrium Hotel Heathrow,Triple Room,3,4,Great south-west road Feltham,"additional toilet,air conditioning,alarm clock...","Room size: 21 m²/226 ft², Shower, 3 single beds"
64170,Gatwick Cambridge Hotel,Triple Room,3,4,"19 Bonehurst Road, Horley","air conditioning,alarm clock,carpeting,coffee/...","Room size: 12 m²/129 ft², Non-smoking, Shower ..."
28925,MStay Russell Court Hotel,Triple,3,4,"9 Russell Road, Kensington","air conditioning,blackout curtains,closet,coff...","Room size: 20 m²/215 ft², City view, Non-smoki..."


# **Evaluasi**

melakukan replace pada value roomamenities dengan sparation ; :  ,

In [101]:
hotel['roomamenities']=hotel['roomamenities'].str.replace(': ;',',')

Membuat Evaluasi berdasarkan kota, rating dan feature hotel

In [102]:
def rekomendasi(city,number,features):
    hotel['city']=hotel['city'].str.lower()
    hotel['roomamenities']=hotel['roomamenities'].str.lower()
    features=features.lower()
    features_tokens=word_tokenize(features)  
    sw = stopwords.words('english')
    lemm = WordNetLemmatizer()
    f1_set = {w for w in features_tokens if not w in sw}
    f_set=set()
    for se in f1_set:
        f_set.add(lemm.lemmatize(se))
    reqbased=hotel[hotel['city']==city.lower()]
    reqbased=reqbased[reqbased['guests_no']==number]
    reqbased=reqbased.set_index(np.arange(reqbased.shape[0]))
    l1 =[];l2 =[];cos=[];
    #print(reqbased['roomamenities'])
    for i in range(reqbased.shape[0]):
        temp_tokens=word_tokenize(reqbased['roomamenities'][i])
        temp1_set={w for w in temp_tokens if not w in sw}
        temp_set=set()
        for se in temp1_set:
            temp_set.add(lemm.lemmatize(se))
        rvector = temp_set.intersection(f_set)
        #print(rvector)
        cos.append(len(rvector))
    reqbased['similarity']=cos
    reqbased=reqbased.sort_values(by='similarity',ascending=False)
    reqbased.drop_duplicates(subset='hotelcode',keep='first',inplace=True)
    return reqbased[['hotelname','roomtype','guests_no','starrating','address','roomamenities','ratedescription','similarity']].head(10)

Mendownload library nltk

In [103]:
nltk.download("punkt")
nltk.download("stopwords")
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

Menampilkan evaluasi berdasarkan kota,rating dan feature yang di inginkan oleh pelanggan

In [104]:
rekomendasi('London', 4, 'I need hot water.')

,hotelname,roomtype,guests_no,starrating,address,roomamenities,ratedescription,similarity
164,Best Western Swiss Cottage,Family Room,4,3,4 Adamson Road,"air conditioning,alarm clock,carpeting,closet,...","Room size: 25 m²/269 ft², Non-smoking, Shower,...",1
694,The Lord Lister Hotel,Family Room,4,3,1 Park Street,"air conditioning,alarm clock,carpeting,closet,...","Room size: 21 m²/226 ft², Shower and bathtub, ...",1
371,Hallmark Hotel London Chigwell Prince Regent,Executive Family Room,4,4,Manor Road,"air conditioning,carpeting,closet,clothes rack...","Non-smoking, Shower and bathtub, 1 double bed",1
417,The Park City Grand Plaza Kensington Hotel,Family Suite,4,4,18-30 Lexham Gardens,"air conditioning,alarm clock,blackout curtains...","Room size: 31 m²/334 ft², City view, Non-smoki...",1
116,Regency House Hotel,Family Room - 3 Pax,4,3,71 Gower Street,"air conditioning,carpeting,closet,coffee/tea m...","Room size: 23 m²/248 ft², Non-smoking, Shower,...",1
735,The Level at Melia White House,The Level Family Room,4,4,Albany Street,"air conditioning,bathrobes,blackout curtains,c...","Shower and bathtub, Executive lounge access, 1...",1
698,Radisson Blu Edwardian Kenilworth - Bloomsbury,Family Room 2 adults + 2 children,4,4,97 Great Russell Street,"air conditioning,alarm clock,bathrobes,carpeti...","Room size: 20 m²/215 ft², Non-smoking, Shower ...",1
130,Mowbray Court Hotel,Family Room,4,3,28/32 Penywern Road,"additional bathroom,air conditioning,alarm clo...","Room size: 20 m²/215 ft², Non-smoking, Shower,...",1
133,The Z Hotel City,Family - 2 Queen Beds,4,4,23-28 Fleet Street,"air conditioning,closet,clothes rack,coffee/te...","Room size: 14 m²/151 ft², City view, Non-smoki...",1
705,Thistle London Heathrow Terminal 5,Family Room,4,3,"Bath Road, Longford, West Drayton. For Sat Nav...","air conditioning,carpeting,closet,coffee/tea m...","Room size: 14 m²/151 ft², Non-smoking, Separat...",1
